# Resources
* [AI-Scientist](https://github.com/SakanaAI/AI-Scientist )
* [LangChain + Neo4j GRG Tutorial](https://python.langchain.com/docs/tutorials/graph/)

# Preliminaries + Installs

These instructions are for Python 3.10
### Install Notebook 04 Dependencies!
* `sudo apt install docker docker.io`
* `sudo chmod 666 /var/run/docker.sock`
* `python3.10 -m pip install docker --user`
### Install ArangoDB + Docker Container
* `sudo docker pull arangodb pyArango`
* `python3.10 -m pip install python-arango adb-cloud-connector --user`



# Init + Env

In [4]:
from os import path, makedirs, environ
from utils import copy_pdfs

environ["_GRG_MODEL_NAME"] = "llama3.2-vision"
environ["_GRG_EMBED_NAME"] = "all-minilm"

environ["_GRG_GRAPH_DB"] = "grg_rel"


# Connect to Graph Database

In [2]:
from time import sleep

import subprocess

def start_arango_container():
    command = [ "docker", "run", "-p", 
                "8529:8529", "-e", "ARANGO_ROOT_PASSWORD=", "arangodb/arangodb"]  
    subprocess.Popen( command )

start_arango_container()
sleep( 15.0 )

Initializing root user...Hang on...
2024-10-31T17:18:44.854457Z [8-2] INFO [a1c60] {syscall} file-descriptors (nofiles) hard limit is 1048576, soft limit is 1048576
2024-10-31T17:18:44.873779Z [8-2] INFO [f6e0e] {aql} memory limit per AQL query automatically set to 20161393459 bytes. to modify this value, please adjust the startup option `--query.memory-limit`
2024-10-31T17:18:48.267555Z [8-2] INFO [c1b63] {arangosearch} ArangoSearch maintenance: [1..1] commit thread(s), [1..1] consolidation thread(s). ArangoSearch execution parallel threads limit: 16
2024-10-31T17:18:48.341856Z [8-2] INFO [7da27] {startup} server will now shut down due to upgrade, database initialization or admin restoration.
Initializing database...Hang on...
Database initialized...Starting System...
2024-10-31T17:18:54.767915Z [1-2] INFO [e52b0] {general} ArangoDB 3.12.3 [linux] 64bit, using jemalloc, build refs/tags/v3.12.3 a80fc5d849b, VPack 0.2.1, RocksDB 7.2.0, ICU 64.2, V8 12.1.165, OpenSSL 3.3.2 3 Sep 2024, bu

In [6]:
# Instantiate ArangoDB Database
import json

from adb_cloud_connector import get_temp_credentials
from arango import ArangoClient

con = get_temp_credentials()

db = ArangoClient(hosts=con["url"]).db(
    con["dbName"], con["username"], con["password"], verify=True
)

print(json.dumps(con, indent=2))

Success: reusing cached credentials
{
  "dbName": "TUTnzdbwn6p7jsgucqg5nhc",
  "username": "TUTbqm7e31brvii5o03o9g0es",
  "password": "TUT757ryealxf8o25bhpspvr",
  "hostname": "tutorials.arangodb.cloud",
  "port": 8529,
  "url": "https://tutorials.arangodb.cloud:8529"
}


In [7]:
# Instantiate the ArangoDB-LangChain Graph
from langchain_community.graphs import ArangoGraph

graph = ArangoGraph( db )

In [8]:
if not db.has_graph( environ["_GRG_GRAPH_DB"] ):
    db.create_graph(
        environ["_GRG_GRAPH_DB"],
        edge_definitions=[
            {
                "from_vertex_collections": ["subjects"],
                "edge_collection": "verbs",
                "to_vertex_collections": ["subjects"],
            },
        ],
    )

In [14]:
import os

import docker

client     = docker.from_env()
containers = client.containers.list()

for container in containers:
    print(container.name, container.short_id, container.status)

os.system( "docker stop 8ed55910d8cc" )

2024-10-31T17:32:50.108624Z [1-2] INFO [b4133] {general} SIGTERM received (sender pid 0), beginning shut down sequence
2024-10-31T17:32:51.113584Z [1-2] INFO [4bcb9] {general} ArangoDB has been shut down
8ed55910d8cc


0